# 품질검사전문기관 목록

In [8]:
from common import commonFunc as cf
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질검사전문기관 목록"
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/품질검사전문기관 목록/selectIoApiPmQtscList.csv'

In [9]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [10]:
JSONKEY = "items"
DUMMY = 0

In [11]:
# 페이지번호 설정
PAGEYN=1
if REQPARAM.count("pageNo") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
### 기본정보 설정 및 파라미터 설정 ###
PAGEYN

1

In [12]:
resultDfMerged = pd.DataFrame()

In [13]:
numOfRows = 1000
resultDf = pd.DataFrame()
pageList = [] # 4/28 추가
APICALL = 0 #4/28 추가
pageNo = 1
# 2022.04.01. 541개 정보가 있으며, 그 이후로는 중복/반복 정보만 서버에서 리턴하고 있음 
# totalCount를 서버에서 리턴하지 않으므로 전체 건수를 알 수 없음. 
# 따라서, PAGEYN을 0으로 고정해서 몇 번 반복 호출해봐야 함
PAGEYN = 0
while True:
    if pageNo >= 5: 
        break
    BASEPARAM={"serviceKey":APIKEY[0], "pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
    #print("BASEPARAM ",BASEPARAM)
    scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, APIKEY,APICALL, JSONKEY, DUMMY)
    resultDf = scrapyResult[0]
    pageList.append( scrapyResult[1] )
    APICALL = scrapyResult[2] 
    if resultDf.empty: # 정상 데이터가 없는 경우
        print("No Data")
        break
    else:
        resultDfMerged = resultDfMerged.append(resultDf)
        pageNo += 1

1 page scraping start apicall iter: 1 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
dataframe품질검사전문기관 목록, param:{'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'} rows: 9 completed
1 page scraping start apicall iter: 2 / used 0974320D-C689-4543-8464-10DE1E5505A9
0 no pageNo
dataframe품질검사전문기관 목록, param:{'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'pageNo': 2, 'numOfRows': 1000, 'type': 'json'} rows: 9 completed
1 page scraping start apicall iter: 3 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
dataframe품질검사전문기관 목록, param:{'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'pageNo': 3, 'numOfRows': 1000, 'type': 'json'} rows: 9 completed
1 page scraping start apicall iter: 4 / used 0974320D-C689-4543-8464-10DE1E5505A9
0 no pageNo
dataframe품질검사전문기관 목록, param:{'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'pageNo': 4, 'numOfRows': 1000, 'type': 'json'} rows: 9 completed


In [14]:
resultDfMerged = resultDfMerged.drop_duplicates()
resultDfMerged

,rgsseq,perno,rgsdt,tpincts,bsnrngcts,mtlnm,rprnnm,tlno,bsnprstcdnm
0,2015056,서울-4-3호,20160622,None,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",(재)한국건설품질연구원,김인식,02-501-5561,이관
1,2015090,인천-3-2호,20151110,품질검사(일반,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",(재)한국화학융합시험연구원 인천,권오정,032-5783-6800,이관
2,2013006,제2013-01호,20130603,특수,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",(주)건설재료시험연구소,남궁 혁,043-716-1122,등록취소
3,2019010,충북-3-12호,20150825,품질검사(일반,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",(주)건설재료시험연구소,성시완,043-716-1122,이관
4,2015059,충북-3-3호,20150825,품질검사(토목,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",(주)건설재료시험연구소,성시완,043-716-1122,이관
...,...,...,...,...,...,...,...,...,...
536,2022006,부산-3-17호,20220302,품질검사(특수),"용접(방사선비파괴검사),용접(자기비파괴검사),용접(초음파비파괴검사),용접(침투비파괴검사)",케이티이 주식회사,김윤길,051-207-0715,이관
537,2022007,인천-3-5호,20150324,품질검사(특수),"용접(방사선비파괴검사),용접(자기비파괴검사),용접(초음파비파괴검사),용접(침투비파괴검사)",에이원기술검사(주),박덕영,032-715-7087,이관
538,2022008,서울-3-32호,20220317,품질검사(특수),"용접(방사선비파괴검사),용접(자기비파괴검사),용접(초음파비파괴검사),용접(침투비파괴검사)",주식회사 올네이션,김종호,070-4346-3110,이관
539,2022010,부산-3-14호,20141002,품질검사(일반,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",(주)한국품질연구원,이미지,051-941-6461,이관


In [13]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

품질검사전문기관 목록 save compled


In [15]:
resultDfMerged.shape

(541, 9)

In [16]:
# 파라미터 저장 : 요청 키가 없는 API이므로, pageList만 저장

# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
# ptlcmnoList = orgParam + ptlcmnoList
# 파라미터 없는경우 ptlcmnoList = []
ptlcmnoList = []
paramList = [ptlcmnoList,pageList]
# # 최종 파라미터 저장
cf.saveparam(paramList, SITENAME,DATANAME,SERVICENAME)